Importing Libraries

In [19]:
import tensorflow as tf

Preprocessing

In [20]:
training_set = tf.keras.utils.image_dataset_from_directory(directory='dataset/training_set', image_size=(64,64), batch_size = 32, label_mode='binary') #use label_mode = 'int' for multiple categories

test_set = tf.keras.utils.image_dataset_from_directory(directory='dataset/test_set', image_size=(64,64), batch_size=32, label_mode='binary')

data_augmentation_train = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomShear(0.2)
])

data_augmentation_test = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255)
])

training_set_augmented = training_set.map(lambda x, y: (data_augmentation_train(x, training=True), y))
test_set_augmented = test_set.map(lambda x, y: (data_augmentation_test(x, training=False), y))

Found 8000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


Building the CNN

In [21]:
cnn = tf.keras.models.Sequential([
    #input_shape would be 64, 64, 1 for black white and 3 for RGB
    #input shape is added only for the first layer
    tf.keras.layers.Conv2D(input_shape=(64, 64, 3), filters=32, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dense(units=1, activation='sigmoid')  # Use 'softmax' for multi-class classification
])
cnn.summary()

Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_10 (Conv2D)              │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 29, 29, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 128)            │       802,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 813,217 (3.10 MB)

 Trainable params: 813,217 (3.10 MB)

 Non-trainable params: 0 (0.00 B)

Compiling and Training the CNN

In [22]:
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])  # Use 'categorical_crossentropy' for multi-class classification
cnn.fit(training_set_augmented, epochs=25, validation_data=test_set_augmented)

Epoch 1/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 28ms/step - accuracy: 0.5391 - loss: 0.7097 - val_accuracy: 0.6800 - val_loss: 0.6211
Epoch 2/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 7s 28ms/step - accuracy: 0.6472 - loss: 0.6313 - val_accuracy: 0.6675 - val_loss: 0.6146
Epoch 3/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 7s 28ms/step - accuracy: 0.6833 - loss: 0.6084 - val_accuracy: 0.6635 - val_loss: 0.6380
Epoch 4/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 7s 28ms/step - accuracy: 0.6989 - loss: 0.5786 - val_accuracy: 0.7280 - val_loss: 0.5448
Epoch 5/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 30ms/step - accuracy: 0.7220 - loss: 0.5474 - val_accuracy: 0.7440 - val_loss: 0.5163
Epoch 6/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - accuracy: 0.7400 - loss: 0.5241 - val_accuracy: 0.7510 - val_loss: 0.5157
Epoch 7/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - accuracy: 0.7465 - loss: 0.5091 - val_accuracy: 0.7655 - val_loss: 0.4955
Epoch 8/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 32ms/step - accuracy: 0.7591 - loss: 0.4909 - val_accu

In [23]:
cnn.save('image_recognition_cnn.keras')  # Save the model

Testing on single data

In [ ]:
training_set.class_names # 0 correspomd to cat and 1 corresponds to dog

['cats', 'dogs']

In [30]:
import numpy as np
test_image = tf.keras.utils.load_img('dataset/single_prediction/cat_or_dog_2.jpg', target_size=(64, 64))
test_image = tf.keras.utils.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)  # Add batch dimension as the model takes images in batch size of 32 images
result = cnn.predict(test_image)
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'
print(f'The predicted class is: {prediction}')  # Output the prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
The predicted class is: cat
